This notebook annotates all gnomad variants with their corresponding malinouis predictions, the phyloP scores associated with their genomic locations, and the genomic regions (enhancer or not) they fall within. 

(Execute notebook after crunching wig into csv, as per other file)

## Import relevant libraries

In [24]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType
import pyspark.sql.functions as F

## Create spark session

In [25]:
if 'spark' in locals() and spark!=None:
    spark.stop()

    #are we running the actual script, or just testing?
for_real=False

spark=None

spark = SparkSession.builder \
    .appName("ANNOTATE") \
    .config("spark.executor.memory", "1900g") \
    .config("spark.executor.cores", "22") \
    .config("spark.driver.memory", "16g") \
    .getOrCreate()

## load in data

In [26]:

#define the phylop tsv schema
phylop_schema = StructType([
    StructField("CHROM", StringType(), True),
    StructField("POS", IntegerType(), True),
    StructField("P_ANNO", FloatType(), True),
])

#read in the phylop tsv
phylop_anno = spark.read \
    .option("comment", "#") \
    .option("delimiter", "\t") \
    .schema(phylop_schema) \
    .csv("/home/mcn26/varef/scripts/noon_data/1.annotate/out_processed.tsv", header=False)

#define the vcf schema
vcf_schema = StructType([
    StructField("CHROM", StringType(), True),
    StructField("POS", IntegerType(), True),
    StructField("ID", StringType(), True),
    StructField("REF", StringType(), True),
    StructField("ALT", StringType(), True),
    StructField("QUAL", StringType(), True),
    StructField("FILTER", StringType(), True),
    StructField("INFO", StringType(), True),

])

#read in the vcf data
vcf = spark.read \
    .option("comment", "#") \
    .option("delimiter", "\t") \
    .schema(vcf_schema) \
    .csv("/gpfs/gibbs/pi/reilly/VariantEffects/scripts/noon_data/0.merge/out/*.vcf.gz", header=False)


#define genomic region annotation (bed file) schema
bed_schema = StructType([
    StructField("CHROM", StringType(), True),
    StructField("START", IntegerType(), True),
    StructField("STOP", StringType(), True),
])

#load the encode enhancer datasets

CRE_BASEPATH="/home/mcn26/varef/data/ENCODE/SCREEN_v4_cCREs_agnostic/"

promoters=spark.read \
    .schema(bed_schema) \
    .option("delimiter", "\t") \
    .csv(CRE_BASEPATH+"GRCh38-PLS.V4.bed.gz")


prox_enhancers=spark.read \
    .schema(bed_schema) \
    .option("delimiter", "\t") \
    .csv(CRE_BASEPATH+"GRCh38-pELS.V4.bed.gz")

distal_enhancers=spark.read \
    .schema(bed_schema) \
    .option("delimiter", "\t") \
    .csv(CRE_BASEPATH+"GRCh38-dELS.V4.bed.gz")

## Split relevant information from the VCF info field

In [27]:
####The `INFO` field contains a lot of useful information, but it is all smashed together into a string. 
#Let's extract information from that string. 

keys_to_extract = [#NONE CAN BE SUBSTRINGS OF THE OTHERS
    "K562__ref", "HepG2__ref", "SKNSH__ref", "K562__alt", "HepG2__alt", "SKNSH__alt",
    "K562__skew", "HepG2__skew", "SKNSH__skew", "AC", "AN", "AF", "cadd_phred",# "P_ANNO" already in its own column
]

# Apply the regexp_extract function to the DataFrame to create new columns for each key.
# The expression '([^;]*)' captures any sequence of characters that are not a semicolon,
# which is assumed to be the delimiter for the key-value pairs in the 'INFO' column.

for key in keys_to_extract:

    #df = df.withColumn(key, regexp_extract(col("INFO"), "{}=([^;]+);?".format(key), 1))
    #when we find something put it, whne we don't put None
    vcf = vcf.withColumn(key, 
                       F.when(
                           F.regexp_extract(F.col("INFO"), "{}=([^;]+);?".format(key), 1) != "",
                           F.regexp_extract(F.col("INFO"), "{}=([^;]+);?".format(key), 1)).otherwise(None))

## Filter out the gorp

- Filter out the gorp
    - Remove low-quality variants that don't pass GNOMAD's own filters.
    - Remove low-quality variants not queried in a large number of individuals
    - Remove variants with a MAF of 0 (they don't really "vary") in the population if they dont exist.
    - Remove those variants that don't have all of the relevant metrics.

In [28]:
vcf = vcf.filter(
    #make sure we have the necessary population stats
    (F.col("AF").isNotNull()) & 
    (F.col("AC").isNotNull()) & 
    (F.col("AN").isNotNull()) & 
    
    #make sure we a cadd_phred score
    (F.col("cadd_phred").isNotNull()) & 

    #check variant has been queried in a reasonably large number of people
    #same as gnomad's own warning threshold
    (F.col("AN").cast("int") > 38077) &
    
    #gnomad filters passed. See original gnomad vcf header for spec.
    (F.col("FILTER") == "PASS") 
    
    #(col("CHROM") == "chr22") &
)

## Add the genomic region annotations

In [29]:
def add_genomic_annotation(loci,regions,name):
    

    # as usual, we have to worry about coordinate systems
    # VCFs are 1-based
    # BEDs are 0-based
    # Chr1        T   A   C   G   T
    #           | | | | | | | | | |
    # 1 based   | 1 | 2 | 3 | 4 | 5
    # 0 based   0   1   2   3   4

    result = loci.join(
        regions,
        (loci.CHROM == regions.CHROM) & 
        (loci.POS > regions.START) & 
        (loci.POS <= regions.STOP),
        how='left'
    )

    # Add a boolean column 'is_in_region'
    result = result.withColumn("is_in_"+name, F.col("start").isNotNull())

    # Select only columns from df_loci and the new boolean column
    final_result = result.select(loci["*"], "is_in_"+name)
    
    return final_result

In [30]:
genomic_regions_added=add_genomic_annotation(loci=vcf,regions=promoters,name="promoter")
genomic_regions_added=add_genomic_annotation(loci=genomic_regions_added,regions=prox_enhancers,name="prox_enhancers")
genomic_regions_added=add_genomic_annotation(loci=genomic_regions_added,regions=distal_enhancers,name="distal_enhancers")

## Add the phyloP annotations

In [31]:
phyloP_annotated = genomic_regions_added.join(phylop_anno, on=["CHROM", "POS"], how="inner")
#inner join means we'll drop all those variants without scores...

Compute mean malinouis reference activity and mean malinouis skew

In [32]:
#reference activity

phyloP_annotated = phyloP_annotated.withColumn("K562__ref", F.col("K562__ref").cast("float"))
phyloP_annotated = phyloP_annotated.withColumn("HepG2__ref", F.col("HepG2__ref").cast("float"))
phyloP_annotated = phyloP_annotated.withColumn("SKNSH__ref", F.col("SKNSH__ref").cast("float"))

phyloP_annotated=phyloP_annotated.withColumn("mean_ref", (F.col("K562__ref") + F.col("HepG2__ref") + F.col("SKNSH__ref")) / 3)

#skew
phyloP_annotated = phyloP_annotated.withColumn("K562__skew", F.col("K562__skew").cast("float"))
phyloP_annotated = phyloP_annotated.withColumn("HepG2__skew", F.col("HepG2__skew").cast("float"))
phyloP_annotated = phyloP_annotated.withColumn("SKNSH__skew", F.col("SKNSH__skew").cast("float"))

phyloP_annotated=phyloP_annotated.withColumn("mean_skew", (F.col("K562__skew") + F.col("HepG2__skew") + F.col("SKNSH__skew")) / 3)

## Add rarity codes

In [33]:
# First, convert 'AC' and 'AF' to integer and float types respectively using withColumn
df = phyloP_annotated.withColumn("AC", F.col("AC").cast("int"))
df = df.withColumn("AF", F.col("AF").cast("float"))

# Calculate MAF using the Spark SQL functions
df = df.withColumn("MAF", F.least(F.col("AF"), 1 - F.col("AF")))

# Use the `when` function to apply the conditional logic
df = df.withColumn("category", 
                   F.when((F.col("MAF") == 0.0) | (F.col("AC") == 0), "MAF_OR_AC_IS_ZERO")
                    .when(F.col("AC") == 1, "SINGLETON")
                    .when(F.col("MAF") < 0.01 / 100, "ULTRARARE")
                    .when(F.col("MAF") < 0.1 / 100, "RARE")
                    .when(F.col("MAF") < 1 / 100, "LOW_FREQ")
                    .otherwise("COMMON"))

In [34]:
df.write.csv("/home/mcn26/varef/scripts/noon_data/1.annotate/annotated_output.csv", header=True, mode="overwrite")

23/12/06 13:43:34 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
ERROR:root:KeyboardInterrupt while sending command.               (0 + 5) / 444]
Traceback (most recent call last):
  File "/home/mcn26/.conda/envs/mcn_vareff/lib/python3.10/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/home/mcn26/.conda/envs/mcn_vareff/lib/python3.10/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/home/mcn26/.conda/envs/mcn_vareff/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 